In [1]:
import numpy as np
import pandas as pd
import random as rnd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
fi = open('keptcolumns.txt', 'r')
keptcolumns = fi.read().rstrip('\n').split(', ')
data = pd.read_csv('epi_r.csv', usecols = keptcolumns)
data = data[pd.notnull(data['calories'])]
data = data.values
h_val = data[:, 0:5]
ingred = data[:, 5:]

In [3]:
def gen_dat(b_vec):
    one_cnt = np.sum(b_vec == 1)
    one_idx = np.where(b_vec == 1)

    n = one_cnt // 2

    x = np.zeros((n, b_vec.shape[0]))
    y = np.zeros((n, b_vec.shape[0]))

    for i in range(n):
        x_size = rnd.randint(n, one_cnt - 1)
        x_one_idx = np.random.choice(one_idx[0], x_size, replace = False)
        y_one_idx = np.setdiff1d(one_idx[0], x_one_idx)
        x[i, x_one_idx] = 1
        y[i, y_one_idx] = 1

    return x, y

In [4]:
x_aug = np.empty((0, ingred.shape[1]))
y_aug = np.empty((0, ingred.shape[1]))
for i in range(ingred.shape[0]):
    x, y = gen_dat(ingred[i])
    x_aug = np.append(x_aug, x, axis = 0)
    y_aug = np.append(y_aug, y, axis = 0)
print(x_aug.shape)
print(y_aug.shape)

(27486, 308)
(27486, 308)


In [5]:
# partition = x_aug.shape[0] * 8 // 10
# x_aug_train = x_aug[0:partition, :]
# y_aug_train = y_aug[0:partition, :]
# x_aug_test = x_aug[partition:, :]
# y_aug_test = y_aug[partition:, :]
x_aug_train = x_aug
y_aug_train = y_aug

In [194]:
model = Sequential([Dense(200, input_shape=(308,)),
                                    LeakyReLU(),
                                    
                                    Dense(100),
                                    LeakyReLU(),
                                    
                                    Dense(100),
                                    LeakyReLU(),
                                                                                                            
                                    Dense(100),
                                    LeakyReLU(),
                                    
                                    Dense(100),
                                    LeakyReLU(),
                                    
                                    Dense(308, activation='sigmoid')
                                   ])

model.compile(loss='binary_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_85 (Dense)             (None, 200)               61800     
_________________________________________________________________
leaky_re_lu_71 (LeakyReLU)   (None, 200)               0         
_________________________________________________________________
dense_86 (Dense)             (None, 100)               20100     
_________________________________________________________________
leaky_re_lu_72 (LeakyReLU)   (None, 100)               0         
_________________________________________________________________
dense_87 (Dense)             (None, 100)               10100     
_________________________________________________________________
leaky_re_lu_73 (LeakyReLU)   (None, 100)               0         
_________________________________________________________________
dense_88 (Dense)             (None, 100)             

In [195]:
history = model.fit(x_aug_train, y_aug_train, epochs=5, batch_size=128)
# loss_and_metrics = model.evaluate(x_aug_test, y_aug_test, batch_size=128)

Epoch 1/5
27486/27486 [==============================] - 2s 58us/step - loss: 0.0869
Epoch 2/5
27486/27486 [==============================] - 1s 42us/step - loss: 0.0324
Epoch 3/5
27486/27486 [==============================] - 1s 42us/step - loss: 0.0324
Epoch 4/5
27486/27486 [==============================] - 1s 42us/step - loss: 0.0318
Epoch 5/5
27486/27486 [==============================] - 1s 42us/step - loss: 0.0309


In [198]:
x_aug_test = np.zeros((1, ingred.shape[1]))
idx = [8, 46, 219, 289]
x_aug_test[0][idx] = 1
y_aug_test = model.predict(x_aug_test, batch_size=128)
idx = np.argsort(y_aug_test[0])
idx = idx[-10:]
idx = idx + 5
print(keptcolumns[idx])

['potato' 'bell pepper' 'salad' 'garlic' 'soup/stew' 'pepper' 'herb'
 'pasta' 'onion' 'tomato']


In [167]:
x = np.array(np.where(ingred[15] == 1)) + 5
print(x - 5)
print(keptcolumns[x])

[[  8  46 219 230 289]]
[['asparagus' 'capers' 'potato' 'radish' 'tuna']]


In [112]:
keptcolumns

array(['rating', 'calories', 'protein', 'fat', 'sodium', 'almond',
       'anchovy', 'apple', 'apple juice', 'apricot', 'artichoke',
       'arugula', 'asian pear', 'asparagus', 'avocado', 'bacon', 'banana',
       'barley', 'basil', 'bass', 'bean', 'beef', 'beef rib',
       'beef shank', 'beef tenderloin', 'beer', 'beet', 'bell pepper',
       'berry', 'biscuit', 'blackberry', 'blue cheese', 'blueberry',
       'bourbon', 'brandy', 'bread', 'breadcrumbs', 'brie', 'brine',
       'brisket', 'broccoli', 'brown rice', 'brownie', 'brussel sprout',
       'butter', 'buttermilk', 'butternut squash', 'butterscotch/caramel',
       'cabbage', 'candy', 'cantaloupe', 'capers', 'caraway', 'cardamom',
       'carrot', 'cashew', 'casserole/gratin', 'cauliflower', 'caviar',
       'celery', 'chard', 'cheddar', 'cheese', 'cherry', 'chestnut',
       'chicken', 'chickpea', 'chile pepper', 'chili', 'chocolate',
       'cilantro', 'cinnamon', 'citrus', 'clam', 'clove',
       'cobbler/crumble', 'cocon

In [164]:
print(h_val[15])

[  5. 421.  10.  33. 383.]
